In [ ]:
!pip install easyocr transformers torch sklearn pillow matplotlib
import easyocr
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle
ocr_reader = easyocr.Reader(['en'])
from google.colab import files
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
print("Uploaded:", image_path)

# Run OCR
results = ocr_reader.readtext(image_path, detail=0)
extracted_text = " ".join(results)

print("\n📌 Extracted Text from Screenshot:\n")
print(extracted_text)
# SENTIMENT
sentiment_model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment"
)
sentiment_tokenizer = AutoTokenizer.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment"
)
sentiment_labels = ["negative", "neutral", "positive"]

# EMOTION
emotion_model = AutoModelForSequenceClassification.from_pretrained(
    "j-hartmann/emotion-english-distilroberta-base"
)
emotion_tokenizer = AutoTokenizer.from_pretrained(
    "j-hartmann/emotion-english-distilroberta-base"
)
emotion_labels = ['admiration','amusement','anger','annoyance','approval','caring',
 'confusion','curiosity','desire','disappointment','disapproval','disgust',
 'embarrassment','excitement','fear','gratitude','grief','joy','love',
 'nervousness','optimism','pride','realization','relief','remorse','sadness',
 'surprise','neutral']
def run_nlp(text, model, tokenizer, labels):
    tokens = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**tokens)
    probs = torch.softmax(outputs.logits, dim=1).detach().numpy()[0]
    return {label: float(prob) for label, prob in zip(labels, probs)}
# Sample training dataset (very small but works)
train_texts = [
    "I love trying new things and exploring ideas",  # Openness
    "I organize everything and plan ahead",          # Conscientiousness
    "I enjoy meeting new people and being social",   # Extraversion
    "I like helping others and being kind",          # Agreeableness
    "I get stressed easily and worry a lot",         # Neuroticism
    "I'm creative and imaginative",                  # Openness
    "I always stick to schedules and rules",         # Conscientiousness
    "I feel energized when I talk to people",        # Extraversion
    "I avoid conflict and try to cooperate",         # Agreeableness
    "Sometimes I feel anxious in situations",        # Neuroticism
]

train_labels = {
    "Openness":           [1,0,0,0,0, 1,0,0,0,0],
    "Conscientiousness":  [0,1,0,0,0, 0,1,0,0,0],
    "Extraversion":       [0,0,1,0,0, 0,0,1,0,0],
    "Agreeableness":      [0,0,0,1,0, 0,0,0,1,0],
    "Neuroticism":        [0,0,0,0,1, 0,0,0,0,1],
}

# Convert text → TFIDF features
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_texts)

# Train 5 models
personality_models = {}
for trait, y in train_labels.items():
    clf = LogisticRegression(max_iter=200)
    clf.fit(X, y)
    personality_models[trait] = clf

print("✅ Local Personality Model Trained Successfully")
def predict_personality(text):
    X_test = vectorizer.transform([text])
    results = {}
    for trait, clf in personality_models.items():
        prob = clf.predict_proba(X_test)[0][1]
        results[trait] = float(prob)
    return results

personality = predict_personality(extracted_text)

print("\n🧬 PERSONALITY (Local Model):")
print(personality)
if extracted_text.strip() == "":
    print("❗ No text detected from screenshot!")
else:
    print("\n🟦 SENTIMENT:")
    sentiment = run_nlp(extracted_text, sentiment_model, sentiment_tokenizer, sentiment_labels)
    print(sentiment)

    print("\n🟧 TOP EMOTIONS:")
    emotion = run_nlp(extracted_text, emotion_model, emotion_tokenizer, emotion_labels)
    top_emotions = dict(sorted(emotion.items(), key=lambda x: x[1], reverse=True)[:5])
    print(top_emotions)
# ---------------------------------------------
# 🔮 AI COUNSELOR SUMMARY (Simple Rule-Based NLP)
# ---------------------------------------------

def ai_counselor_summary(text, personality, sentiment, emotions):
    summary = ""

    # --- Emotional Tone ---
    dominant_emotion = max(emotions, key=emotions.get)
    sentiment_label = max(sentiment, key=sentiment.get)

    summary += f"**Emotional Tone:** The message primarily expresses **{dominant_emotion}**, with an overall **{sentiment_label}** sentiment.\n\n"

    # --- Personality Pattern ---
    sorted_traits = sorted(personality.items(), key=lambda x: x[1], reverse=True)
    top_trait, top_value = sorted_traits[0]

    summary += f"**Personality Indications:** Based on the writing sample, the strongest trait that appears is **{top_trait}**. "

    if top_trait == "Openness":
        summary += "The language suggests creativity, adaptability, and curiosity.\n\n"
    elif top_trait == "Conscientiousness":
        summary += "There are signs of reliability, planning, or a structured communication style.\n\n"
    elif top_trait == "Extraversion":
        summary += "There are elements of sociability, warmth, or outward-focused engagement.\n\n"
    elif top_trait == "Agreeableness":
        summary += "Tone shows cooperation, friendliness, or emotional warmth.\n\n"
    elif top_trait == "Neuroticism":
        summary += "There may be small hints of emotional tension, sensitivity, or uncertainty.\n\n"

    # --- Supportive Guidance ---
    if sentiment_label == "positive":
        summary += "**Support Insight:** The conversation carries a healthy and positive tone. The individual seems comfortable and emotionally balanced in this context."
    elif sentiment_label == "neutral":
        summary += "**Support Insight:** Tone appears calm and neutral. If this is typical communication, they may be emotionally steady but not expressive."
    else:
        summary += "**Support Insight:** There may be some emotional strain or worry behind the message. Gentle reassurance or clearer communication might help."

    return summary

ai_summary = ai_counselor_summary(
    extracted_text,
    personality,
    sentiment,
    top_emotions
)

print("\n🔮 AI COUNSELOR SUMMARY:\n")
print(ai_summary)




  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


Saving msgidea1.png to msgidea1 (9).png
Uploaded: msgidea1 (9).png


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



📌 Extracted Text from Screenshot:

Emma what time is our reservation tonight? Hil I got us a table for 7 p.m: Sounds good! See you soon Hey;


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


✅ Local Personality Model Trained Successfully

🧬 PERSONALITY (Local Model):
{'Openness': 0.19529410219391505, 'Conscientiousness': 0.1950323080636582, 'Extraversion': 0.1900288798178595, 'Agreeableness': 0.19379519971793957, 'Neuroticism': 0.20012746144230764}

🟦 SENTIMENT:
{'negative': 0.0007768501527607441, 'neutral': 0.03093510866165161, 'positive': 0.9682880640029907}

🟧 TOP EMOTIONS:
{'approval': 0.3656977415084839, 'annoyance': 0.3536094129085541, 'confusion': 0.22936080396175385, 'caring': 0.033480361104011536, 'admiration': 0.01010984554886818}

🔮 AI COUNSELOR SUMMARY:

**Emotional Tone:** The message primarily expresses **approval**, with an overall **positive** sentiment.

**Personality Indications:** Based on the writing sample, the strongest trait that appears is **Neuroticism**. There may be small hints of emotional tension, sensitivity, or uncertainty.

**Support Insight:** The conversation carries a healthy and positive tone. The individual seems comfortable and emotion

In [ ]:
whos


Variable                             Type                                Data/Info
----------------------------------------------------------------------------------
AutoModelForSequenceClassification   type                                <class 'transformers.mode<...>rSequenceClassification'>
AutoTokenizer                        type                                <class 'transformers.mode<...>tion_auto.AutoTokenizer'>
Image                                module                              <module 'PIL.Image' from <...>t-packages/PIL/Image.py'>
LogisticRegression                   type                                <class 'sklearn.linear_mo<...>stic.LogisticRegression'>
TfidfVectorizer                      type                                <class 'sklearn.feature_e<...>on.text.TfidfVectorizer'>
X                                    csr_matrix                          <Compressed Sparse Row sp<...>9, 32)	0.4601578921271882
ai_counselor_summary                 function               